In [23]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [24]:
directors = []
mpaaRatings = []
X = [] # This holds the movie data that the model will use to train.
y = [] # This holds whether the movies won an oscar or not that the model will use to train.

dummyData = pd.read_csv("oscarsDummyData.csv", encoding = "ISO-8859-1")
n = len(dummyData.index)

for i in range(0, n):
    movie = list(dummyData.iloc[i])
    director = movie[7]
    mpaaRating = movie[10]
    
    if not(director in directors):
        directors.append(director)
    movie[7] = directors.index(director) # replace director to its unique numerical value
    
    if not(mpaaRating in mpaaRatings):
        mpaaRatings.append(mpaaRating)
    movie[10] = mpaaRatings.index(mpaaRating) # replace mpaa rating to its unique numerical value
    
    X.append(movie[3:-1])
    
    if movie[1]:
        y.append(1)
    else:
        y.append(0)

# train_test_split takes the datasets, shuffles the datasets, and splits it into training and testing sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, stratify = y)

In [25]:
print(X_train)

[[104, 4500000, 255407969, 250907969, 81, 18, 1, 1, 98, 2, 2017, 92], [131, 1700000, 0, -1700000, 6, 17, 9, 1, 93, 1, 2002, 30], [117, 5000000, 55736588, 50736588, 57, 24, 7, 1, 93, 11, 2013, 67], [103, 200000000, 1066969703, 866969703, 47, 15, 12, 3, 98, 6, 2010, 95], [106, 100000000, 525573161, 425573161, 43, 24, 11, 0, 92, 7, 2017, 84], [114, 14000000, 233555708, 219555708, 63, 24, 8, 0, 90, 12, 2014, 74], [116, 47000000, 203388186, 156388186, 72, 19, 3, 0, 94, 11, 2016, 112], [115, 15000000, 160192031, 145192031, 84, 21, 4, 1, 90, 12, 2017, 110], [109, 25000000, 163670000, 138670000, 27, 24, 1, 1, 90, 9, 2009, 78], [144, 108000000, 630161890, 522161890, 3, 13, 2, 0, 91, 10, 2015, 76], [94, 18000000, 35692920, 17692920, 46, 13, 7, 1, 93, 11, 2010, 72], [133, 50000000, 110206216, 60206216, 19, 20, 8, 0, 94, 9, 2011, 66], [170, 110000000, 213719942, 103719942, 9, 24, 10, 0, 86, 12, 2004, 59], [115, 12000000, 17654912, 5654912, 18, 13, 11, 1, 91, 11, 2013, 118], [106, 7000000, 31430334

In [29]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
predictions = classifier.predict(X_test)
pretty_list = "\n".join("{}\t\t{}".format(x, y) for x, y in zip(predictions, y_test))
print("Predictions\tActual")
print(pretty_list)


Predictions	Actual
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
1		1
1		1
0		0
0		0
0		0
0		0
0		0
0		0
0		0
0		0
1		1
0		0
0		0
0		0
1		1
0		0


In [32]:
predictions_percent = classifier.predict_proba(X_test)[:,1]
pretty_list = "\n".join("{:.2f}\t{}".format(x, y) for x, y in zip(predictions_percent, y_test))
print("% of 1 \tActual")
print(pretty_list)

% of 1 	Actual
0.00	0
0.00	0
0.01	0
0.01	0
0.01	0
0.01	0
0.01	0
0.00	0
0.00	0
0.83	1
0.98	1
0.00	0
0.00	0
0.00	0
0.03	0
0.08	0
0.01	0
0.11	0
0.00	0
0.92	1
0.01	0
0.00	0
0.15	0
0.93	1
0.00	0


In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, predictions)
print(cm)
accuracy_score(y_test, predictions)

[[21  0]
 [ 0  4]]


1.0